# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk import word_tokenize
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer  
nltk.download('wordnet')
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,precision_score,recall_score,classification_report
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('Message',con=engine)
X =df.message 
Y =df.drop(columns=['message','id','original','genre'],axis=1) 

In [4]:
#Let us try to see our messages to see how to tokenize them
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [5]:
Y.iloc[:,0:35]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """ 
    A function that takes text as an input,normaliyes and tokenise it to return the tokens 
    as output
    Extended description of function. 
  
    Parameters: 
    text: text data that we have to tokenize 
  
    Returns: 
    tokens from the input text
  
    """
    text=text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text=word_tokenize(text)
    stop_words=stopwords.words('english')
    words = [w for w in text if w not in stop_words]
    words=[]
    for w in text: 
        if w not in stop_words:
            words.append(w) 
    clean_tokens=[]
    lemmatizer = WordNetLemmatizer()
    for w in words:
        clean_tok=lemmatizer.lemmatize(w)
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [7]:
df['message'].apply(tokenize)

0        [weather, update, cold, front, cuba, could, pa...
1                                              [hurricane]
2                                 [looking, someone, name]
3        [un, report, leogane, 80, 90, destroyed, hospi...
4        [say, west, side, haiti, rest, country, today,...
5                          [information, national, palace]
6                            [storm, sacred, heart, jesus]
7                 [please, need, tent, water, silo, thank]
8                   [would, like, receive, message, thank]
9        [croix, de, bouquet, health, issue, worker, sa...
10                [nothing, eat, water, starving, thirsty]
11       [petionville, need, information, regarding, 4636]
12       [thomassin, number, 32, area, named, pyron, wo...
13       [let, together, need, food, delma, 75, didine,...
14       [information, 4636, number, order, participate...
15       [comitee, delmas, 19, rue, street, janvier, im...
16       [need, food, water, klecin, 12, dying, hunger,.

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
Y_pred=pipeline.predict(X_test)
Y_pred_df=pd.DataFrame(Y_pred,columns=Y_test.columns)
for column in Y_test.columns:
    classification_report(Y_test[column],Y_pred_df[column])

TypeError: 'str' object is not callable

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1},
            {'text_pipeline': 0.5},
            {'text_pipeline': 0.8},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.